In [127]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output
import utils
from utils import keras_baselilne
from utils import resnet

In [32]:
#Load data
train = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/train.json")
valid = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train_half/valid.json")
test = pd.read_json("/mnt/extDisk/courses/data/statoil/data/train/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
valid.inc_angle = valid.inc_angle.replace('na', 0)
valid.inc_angle = valid.inc_angle.astype(float).fillna(0.0)
print("done!")

done!


In [30]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])

# Valid data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in valid["band_2"]])
X_valid = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_valid = np.array(valid.inc_angle)
y_valid = np.array(valid["is_iceberg"])

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test = np.array(test.inc_angle)

In [118]:
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
    

model = keras_baselilne()
model.summary()

utils.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
  img_1 = Conv2D(16,3,3, activation=p_activation) (input_1)
utils.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="elu")`
  img_1 = Conv2D(16,3,3, activation=p_activation) (img_1)
utils.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
  img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
utils.py:37: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="elu")`
  img_1 = Conv2D(32, 3,3, activation=p_activation) (img_1)
utils.py:40: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  img_1 = Conv2D(64,3,3, activation=p_activation) (img_1)
utils.py:41: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  img_1 = Conv2D(64,3,3, activation=p_ac

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
X_1 (InputLayer)                 (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
conv2d_169 (Conv2D)              (None, 73, 73, 16)    448         X_1[0][0]                        
____________________________________________________________________________________________________
conv2d_170 (Conv2D)              (None, 71, 71, 16)    2320        conv2d_169[0][0]                 
____________________________________________________________________________________________________
max_pooling2d_106 (MaxPooling2D) (None, 35, 35, 16)    0           conv2d_170[0][0]                 
___________________________________________________________________________________________

In [21]:
model.load_weights(filepath="/mnt/extDisk/courses/data/statoil/data/keras_baseline/.model_weights.hdf5")

print("Train evaluate:")
print(model.evaluate([X_train, X_angle_train], y_train, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate([X_valid, X_angle_valid], y_valid, verbose=1, batch_size=200))

Train evaluate:
600/602 [============================>.] - ETA: 0s[0.23146934386503656, 0.88372094211388263]
####################
watch list evaluate:
200/200 [==============================] - 0s
[0.25437977910041809, 0.89500004053115845]


In [6]:
prediction = model.predict([X_test, X_angle_test], verbose=1, batch_size=200)

8424/8424 [==============================] - 1s     


In [107]:
prediction = pd.read_csv('/mnt/extDisk/courses/data/statoil/data/ensamble/ensamble_0211.csv')

In [140]:
print(prediction.head())
print(len(test))
test_label = pd.merge(test,prediction,on=['id'])
print(len(test_label))
print(len(test_label[(test_label.is_iceberg < 0.01) | (test_label.is_iceberg >0.99)]))
test_label = test_label[(test_label.is_iceberg < 0.01) | (test_label.is_iceberg >0.99)]

x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_label["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test_label["band_2"]])
X_test_label = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_test_label = np.array(test_label.inc_angle)
test_y = np.array(test_label["is_iceberg"])

         id  is_iceberg
0  000c65d7    0.840560
1  001680af    0.209411
2  00193999    0.883449
3  00247e48    0.990012
4  00503faa    0.000414
8424
8424
1663


In [141]:
X_mix = np.concatenate([X_train,X_test_label])
X_angle_mix = np.concatenate([X_angle_train,X_angle_test_label])
y_mix = np.concatenate([y_train,test_y])

In [142]:
#layers = model.layers
# Get the index of the first dense layer...
#first_dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]
# ...and set this and all subsequent layers to trainable
#for layer in layers[first_dense_idx:]: layer.trainable=True

In [145]:
file_path = "/mnt/extDisk/courses/data/statoil/data/pseudo_labeling/.model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

model = utils.resnet()
model.fit([X_mix], y_mix, epochs=25
          , validation_data=([X_valid], y_valid)
         , batch_size=32
         , callbacks=callbacks)

Train on 2265 samples, validate on 200 samples
Epoch 1/25
2265/2265 [==============================] - 16s - loss: 0.6796 - acc: 0.1585 - val_loss: 7.2531 - val_acc: 0.5500
Epoch 2/25
2265/2265 [==============================] - 3s - loss: 0.3533 - acc: 0.1806 - val_loss: 1.0514 - val_acc: 0.6800
Epoch 3/25
2265/2265 [==============================] - 3s - loss: 0.2092 - acc: 0.2009 - val_loss: 0.6251 - val_acc: 0.7200
Epoch 4/25
2265/2265 [==============================] - 3s - loss: 0.1581 - acc: 0.2150 - val_loss: 0.7459 - val_acc: 0.7200
Epoch 5/25
2265/2265 [==============================] - 3s - loss: 0.1407 - acc: 0.2203 - val_loss: 0.5440 - val_acc: 0.8300
Epoch 6/25
2265/2265 [==============================] - 3s - loss: 0.1180 - acc: 0.2256 - val_loss: 1.0412 - val_acc: 0.7450
Epoch 7/25
2265/2265 [==============================] - 3s - loss: 0.1317 - acc: 0.2265 - val_loss: 0.2780 - val_acc: 0.8950
Epoch 8/25
2265/2265 [==============================] - 3s - loss: 0.0859 - a

In [146]:
model.load_weights(filepath=file_path)

print("Train evaluate:")
print(model.evaluate(X_mix, y_mix, verbose=1, batch_size=200))
print("####################")
print("watch list evaluate:")
print(model.evaluate(X_valid, y_valid, verbose=1, batch_size=200))

Train evaluate:
2200/2265 [============================>.] - ETA: 0s[0.066116784845230064, 0.2481236213419348]
####################
watch list evaluate:
200/200 [==============================] - 0s
[0.27802926301956177, 0.89500004053115845]


In [147]:
prediction = model.predict(X_test, verbose=1, batch_size=200)

8400/8424 [============================>.] - ETA: 0s

In [148]:
submission = pd.DataFrame({'id': test["id"], 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submission.head(10)

,id,is_iceberg
0,5941774d,0.050204
1,4023181e,0.557825
2,b20200e4,0.007555
3,e7f018bb,0.999854
4,4371c8c3,0.110107
5,a8d9b1fd,0.175879
6,29e7727e,0.053483
7,92a51ffb,0.999565
8,c769ac97,0.062707
9,aee0547d,0.022476


In [149]:
submission.to_csv("/mnt/extDisk/courses/data/statoil/data/train/pseudo_labeling_sure_resnet.csv", index=False)

<module 'utils' from 'utils.py'>